## CRYSTALS-Kyber: A Baby Kyber implementation on Python 🛸

Hi! this is **Grupo 03**, we see our faces again! (well, actually that's not true, because this is a computer...whatever).

If you enjoyed our first steps, that means you are ready to join us in this new adventure.

Kyber is a Post-Quantum Key-Establishment Mechanism (PQ KEM) base on the Module - Learning with Errors problem

In this notebook we will show you an example of Kyber at an escalated level, called "Baby Kyber".

First, some notation. In Kyber we work with a ring of integer coefficient ($mod$ $q$) and grade $n-1$ polynomials.

Each element on a Kyber vector belongs to $R_{q}$, this being:
$$ R_{q} = \mathbb{Z}_q (X) / (X^n  + 1) $$
For instance, a polynomial $a_{(X)}$ $\in$ $R_{q}$:
$$  a_{(X)} = \sum_{i=1}^{n-1} = a_i X^i = a_0 + a_1 X + a_2 X^2 + \cdots + a_{n-1}X^{n-1} \qquad (a_i \thinspace mod \thinspace q)$$
Any vector in this example will have $k$ elements, this means, a vector $\mathbf{v}$ in this field could be:
$$ \mathbf{v} \in R_{q}^k \qquad \mathbf{v} = [v_1, v_2, \cdots, v_k]^T \qquad v_i \in R_q \quad i = (1,2,\cdots,k)$$

In our baby kyber example we consider:


| **Parameter**  |   **Value** |
|----------------|:-----------:|
|       $n$      |     4       |
| $q$            |     17      |
| $k$            |     2       |

<div style="text-align:center"> note that $q$ is a prime number and chosen to be equal to $2^n + 1$



In [2]:
import numpy as np
import math
q = 17 #module
n = 4 #grade
k = 2 #number of elements

<div style="text-align:center"><img src="attachment:803c0728-8626-4863-b729-3201e7b49b4e.png" width="220" />
</div>

<div style="text-align:center"> Kyber Crystal being forge by Midjourney<div>

Now let's get down to business.

We can split the Kyber KEM in three steps:
 - **Key Generation**
 - **Encryption**
 - **Decription**

In the **Key Generation** step we genete both the public and the secret key $(p_k,s_k)$

$$ p_k: \mathbf{t} = \mathbf{A} \thinspace \mathbf{s} + \mathbf{e}$$
$$ s_k: \mathbf{s}$$

- $\mathbf{A}$ $\in$ $R_q^{k \thinspace x \thinspace k}$ is a polynomial matrix. Each coefficient in $\mathbf{A}$ is uniformly sampled. Kyber standard uses a *PARSE* function to sample this coefficients.


In [3]:
# Generate matrix A from an uniformly random distribution
#---------------------------------------------------------------
A = np.zeros([2, 2, 4], dtype=int)
for i in range(k):
    for j in range(k):
        A[i][j] = np.random.randint(low=0, high=q, size=n)
#  ---------- end of matrix A generation -----------------------
print("Matrix A:")
#print(A)
print("A11:", A[0][0])
print("A12:", A[0][1])
print("A21:", A[1][0])
print("A22:", A[1][1])

Matrix A:
A11: [15 15 14  1]
A12: [13  1  2 14]
A21: [16  7 14  2]
A22: [ 6 15  9 11]


- $\mathbf{s}$ $\in$ $R_q^k$   is the secret key, a noise vector. Noise in Kyber is sample from a Centered Binomial Distribution (CBD).
- $\mathbf{e}$ $\in$ $R_q^k$  is another noise vector, it is the error we add in our M-LWE problem.


In [4]:
# Generate s , e from an CBD
#---------------------------------------------------------------
trials, p = 2, .5  # number of trials, probability of each trial
l = 4 # array lenght
# Noise vector s:
s = np.zeros((k,l), dtype=int)
for j in range(k):
    a = np.random.binomial(trials, p, l)
    b = np.random.binomial(trials, p, l)
    for i in range(l):
        s[j][i] = a[i] - b[i]
# Noise vector e:
e = np.zeros((k,l), dtype=int)
for j in range(k):
    a = np.random.binomial(trials, p, l)
    b = np.random.binomial(trials, p, l)
    for i in range(l):
        e[j][i] = a[i] - b[i]
# ------------- end noise generation -------------------------------
print("Vector s:")
print(s)
print("Vector e:")
print(e)

Vector s:
[[ 1 -1  1  0]
 [ 2 -1  0  0]]
Vector e:
[[-1  1 -2 -1]
 [-1  1 -1  2]]


In [5]:
#First we multiply the verctor s with A:
Axs = np.zeros((k,l), dtype=int)
poly_mod_negacyclic = np.array([1,0, 0, 0, 1])

for i in range(k):
    for j in range(k):
        aux = np.convolve(A[i][j],s[j])
        aux = aux % q
        [quo, rem] = np.polydiv(aux, poly_mod_negacyclic)
        if (len(rem) < l):
            rem = np.concatenate((np.zeros(1),rem))
        rem = rem % q
        Axs[i] = Axs[i] + rem
Axs = Axs % q
#then we add the error e
t = Axs + e
t = t % q

print("A o s:\n",Axs)
print("public key t:\n",t)
print("secret key s:\n",s)

A o s:
 [[11  9 12  0]
 [ 8  7  4  8]]
public key t:
 [[10 10 10 16]
 [ 7  8  3 10]]
secret key s:
 [[ 1 -1  1  0]
 [ 2 -1  0  0]]


So far we manage to accomplish the first step of our implementation, so let's move onto the next step: **encryption**

Encrypting requires the public key $\mathbf{t}$ and a message $m$ as input, an generates the ciphertext $c$ as output. In this simple example, we will use the same matrix $\mathbf{A}$ we sample in the **Key Generation** step, but in real Kyber it is generated freshly as part of every encryption.

Let's say we want to send is the number 9, which can be represented as a 4 bit digit in BCD:

the number 3 in binary is 0011 $\rightarrow$ we encode it as a polynomial array:


In [7]:
q2 = math.ceil(q/2) #module over two ceiled
# ENCODE:
# ----------------------------
message = np.array( [0, 0, 1, 1], dtype=int)
print("message:\n",message)
print("message as a polynomial array:\n",message)
message = message * q2
#------end of encodig---------
print("endoded message:\n",message)

message:
 [0 0 1 1]
message as a polynomial array:
 [0 0 1 1]
endoded message:
 [0 0 9 9]


We also need some noise vectors for the ciphertext generation
- $\mathbf{r}$ $\in$ $R_q^{k}$ is a polynomial noise vector sampled from a CBD:
- $\mathbf{e_1}$ $\in$ $R_q^k$ is a polynomial noise vector sampled from a CBD:
- $\mathbf{e_2}$ $\in$ $R_q$  is a polynomial sampled from a CBD:


In [8]:
# Generate r , e1, e2 from a CBD
#---------------------------------------------------------------
trials, p = 2, .5  # number of trials, probability of each trial
l = 4 # array lenght
# Noise vector s:
r = np.zeros((k,l), dtype=int)
for j in range(k):
    a = np.random.binomial(trials, p, l)
    b = np.random.binomial(trials, p, l)
    for i in range(l):
        r[j][i] = a[i] - b[i]
# Noise vector e:
e1 = np.zeros((k,l), dtype=int)
for j in range(k):
    a = np.random.binomial(trials, p, l)
    b = np.random.binomial(trials, p, l)
    for i in range(l):
        e1[j][i] = a[i] - b[i]
e2 = np.zeros(l, dtype=int)
a = np.random.binomial(trials, p, l)
b = np.random.binomial(trials, p, l)
for i in range(l):
    e2[i] = a[i] - b[i]
# ------------- end noise generation -------------------------------
print("error r:\n", r)
print("error e1:\n", e1)
print("error e2:\n", e2)

error r:
 [[ 1 -1 -2 -1]
 [ 1  0  0  2]]
error e1:
 [[ 0 -1  1  0]
 [ 0 -1 -1 -1]]
error e2:
 [-1 -1  0  0]


Kyber ciphertext consist in two values: $\mathbf{u}$, a polynomial vector in $R_q^{k}$, and $v$ a polynomial in $R_q$.

$$ \mathbf{u} = \mathbf{A}^T \thinspace \mathbf{r} + \mathbf{e_1}  $$

$$ v = \mathbf{t}^T \thinspace \mathbf{r} + e_2 + m  $$

To obtain $\mathbf{u}$ first we transpose the matrix $\mathbf{A}$, multiply it by $\mathbf{r}$

In [ ]:
# Transpose Matrix A:
AT = np.zeros((k,k,l),dtype = int)
print(A)
AT[0][0] = A[0][0]
AT[1][0] = A[0][1]
AT[0][1] = A[1][0]
AT[1][1] = A[1][1]
print("MATRIX A TRANSPOSE")
print("AT11:", AT[0][0])
print("AT12:", AT[0][1])
print("AT21:", AT[1][0])
print("AT22:", AT[1][1])

print(AT)
# Multiply AT with r
ATxr = np.zeros((k,l), dtype=int)
for i in range(k):
    for j in range(k):
        aux = np.convolve(AT[i][j],r[j])
        aux = aux % q
        [quo, rem] = np.polydiv(aux, poly_mod_negacyclic)
        if (len(rem) < l):
            rem = np.concatenate((np.zeros(1),rem))
        rem = rem % q
        ATxr[i] = ATxr[i] + rem
ATxr = ATxr % q
print("A TRANSPOSE by r:\n",ATxr)

[[[10 10  9  1]
  [16 13  8  7]]

 [[ 9  0  2  9]
  [15  9  4 11]]]
MATRIX A TRANSPOSE
AT11: [10 10  9  1]
AT12: [9 0 2 9]
AT21: [16 13  8  7]
AT22: [15  9  4 11]
[[[10 10  9  1]
  [ 9  0  2  9]]

 [[16 13  8  7]
  [15  9  4 11]]]
A TRANSPOSE by r:
 [[ 7  1 10 12]
 [ 5 16  1  5]]


Now, we add the noise vector $\mathbf{e_1}$

In [ ]:
#now we add the error e1
u = ATxr + e1
u = u % q
# ---------- end u ------------------------------
print("u:\n",u)

u:
 [[ 6  3 10 12]
 [ 4 16  3  3]]


To generate te polynomial $v$, we do:
$$ v = \mathbf{t}^T \thinspace \mathbf{r} + e_2 + m  $$

In [ ]:
# Obtain v:
# ----------------------------------------------
v_aux = np.zeros(n, dtype=int)
for j in range(k): # here we multiply t (transpose) by r (RN. Study well this partm in a real-implem this is done by the NTT)
    aux = np.convolve(t[j],r[j])
    aux = aux % q
    [quo, rem] = np.polydiv(aux, poly_mod_negacyclic)
    if (len(rem) < l):
        rem = np.concatenate((np.zeros(1),rem))
    rem = rem % q
    v_aux = v_aux+ rem
v_aux = v_aux % q
# here i've multiply t^T * r, now we add e2
v_aux = v_aux + e2
v_aux = v_aux % q
# finally we add the encoded message
v = v_aux + message
v = v % q
# ---------- end v ------------------------------
print("ciphertext , c = (u,v)")
print("u:\n",u)
print("v:\n",v)

ciphertext , c = (u,v)
u:
 [[ 6  3 10 12]
 [ 4 16  3  3]]
v:
 [ 5. 12. 12.  6.]


Finally we implement the **Decryption** step, where we recover the message by doing the following operation:

$$ m = v - \mathbf{s}^T \thinspace \mathbf{u}$$

In [ ]:
m_noisy = np.zeros(l, dtype=int)
for j in range(k):
    aux = np.convolve(s[j],u[j])
    aux = aux % q
    [quo, rem] = np.polydiv(aux, poly_mod_negacyclic)
    if (len(rem) < l):
        rem = np.concatenate((np.zeros(1),rem))
    rem = rem % q
    m_noisy = m_noisy + rem
m_noisy = m_noisy % q

m_noisy = v - m_noisy
m_noisy = m_noisy % q
print("Noisy message:\n", m_noisy)
#finally we decode the message:
m = np.zeros(l, dtype=int)
for i in range(len(m_noisy)):
    if ( math.floor(q/4) < m_noisy[i] <= math.floor((q*(3/4)))): # here we map the resutl to be binary (0 or 1)
        m[i] = 1
    else:
        m[i] = 0
print("message:\n", m)

Noisy message:
 [11.  0.  5.  6.]
message:
 [1 0 1 1]


## Appendix A: convolutions and transforms 🐛 ➡ 🦋

A core concept in our implmentation is **convolution**.

Let $a_{[k]}$ and $b_{[k]}$ be two finite discrete sequences of length $n$, such that:

$$ a[k] = (a[0], a[1],a[2], \cdots,a[n-1]) \qquad b[k] = (b[0], b[1],b[2], \cdots,b[n-1]) $$

The **Lineal** or **acyclic** convolution between this sequences is definded as:

$$ \tilde{z[k]} =  \sum_{k=1}^{n-1} a[i] b[k-i]  $$

Where $\tilde{z}[k]$ is a sequence of length $2n -1$ (this means it has $2n -2 $ elements).

On the one hand the **cyclic** convolution of two sequences of length $n$, $a[k]$ and $b[k]$, can be defined as another length $n$ sequence $z[k]$ such that:

$$ z[k] = \tilde{z}[k] + \tilde{z}[k + n] \qquad where \qquad \thinspace \tilde{z}[2n - 1] = 0$$

On the other hand, the **nega-cyclic** convolution can be defined as another lentgh $n$ sequence $y[k]$ being this one:

$$ y[k] = \tilde{z}[k] - \tilde{z}[k + n] \qquad where \qquad \thinspace \tilde{z}[2n - 1] = 0$$$$


Now, let $A(x)$ and $B(x)$ two polynomials of degree at most $n - 1 $ where:

$$ A(x) =  \sum_{k=1}^{n-1} a[k] x^k \qquad B(x) =  \sum_{k=1}^{n-1} b[k] x^k $$

It is easy to proof that the linear convolution between $ a[k]$ and $b[k]$ hols the multiplication of the polynonials $A(x)$ and $B(x)$

$$ A(x)B(x) = \sum_{k=1}^{2n-2} \tilde{z}[k] x^k $$

